In [1]:
from google.colab import files
files.upload() # kaggle.jsonをアップロード
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
# 相互情報量を計算するメソッドをインポート
from sklearn.feature_selection import mutual_info_classif

# データの準備
def prepare():
    !kaggle datasets download -d pmenshih/kpmi-mbti-mod-test 
    !unzip kpmi-mbti-mod-test.zip

# 前処理（使用する項目の絞り込み、正規化）
def preprocess():
    # MBTI診断データを読込み
    df = pd.read_csv('kpmi_data.csv', sep=';')
    # 現在の職業に満足しているかどうか(yes:1, no:0)
    y = df.loc[:, 'satisfied'].values
    # 使用する特徴量（MBTIのスコア)
    scales = ['scale_e','scale_i','scale_s','scale_n',
              'scale_t','scale_f','scale_j','scale_p']
    df = pd.DataFrame(df.loc[:,scales], columns=scales)
    print(df)
    X = df.loc[:, df.columns.values].values
    X_train, X_test, y_train, y_test = train_test_split(X, y,
   random_state=0, train_size=0.9)
    return X_train, X_test, y_train, y_test, scales

# ランダムフォレストで分類評価
def predict_satisfaction(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(max_depth=4, random_state=2)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('Accuracy = {:.3f}'.format(accuracy_score(
  y_test, y_pred)))
    labels = ['no', 'yes']
    print(classification_report(y_test, y_pred,
   target_names=labels))
    
# MI (相互情報量)を用いて特徴選択
def select_feature_by_MI(X_train, X_test, y_train, y_test, scales, n_features): 
    # n_featuers 個の特徴量を選択
    selecter = SelectKBest(
        mutual_info_classif, k=n_features).fit(
                     X_train, y_train)

    sel_features = []
    selected_feature = selecter.get_support()
    for i in range(len(selected_feature)):
        if selected_feature[i]:
            print('Selected Feature - {}'.format( \
                                          scales[i])) 
            sel_features.append(scales[i])
    trdf = pd.DataFrame(X_train, columns=scales) 
    tedf = pd.DataFrame(X_test, columns=scales)
    X_train = trdf.loc[:, sel_features].values
    X_test = tedf.loc[:, sel_features].values
    return X_train, X_test

def main():
    prepare()
    X_train, X_test, y_train, y_test, scales = preprocess()
    print('- 特徴選択無し [%d個の特徴量] -' % len(scales))
    predict_satisfaction(X_train, X_test, y_train, y_test)
    n_features = 3
    print('- MIによる特徴選択 [%d個の特徴量] -' % n_features)
    X_train, X_test = select_feature_by_MI(X_train, X_test,
   y_train, y_test, scales, n_features)
    predict_satisfaction(X_train, X_test, y_train, y_test)

if __name__ == '__main__':
    main()


  0% 0.00/985k [00:00<?, ?B/s]
100% 985k/985k [00:00<00:00, 67.0MB/s]
Archive:  kpmi-mbti-mod-test.zip
  inflating: kpmi_data.csv           
  inflating: kpmi_key.json           
  inflating: questionnaire_schema.json  


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


       scale_e  scale_i  scale_s  scale_n  scale_t  scale_f  scale_j  scale_p
0            5       26       25       12       24        8       27       10
1           16       22       17       16       23       11       22       15
2           28        6       14       18       20       16       20       15
3           28       10       22       10       16       14       22       12
4           24       10       16       17       13       23       31        3
...        ...      ...      ...      ...      ...      ...      ...      ...
21841       25       15       17       20       13       13       19       15
21842       26        6       20       18       11       21       22       11
21843       16       21       22       11       28        5       25        9
21844       21       13       12       17        8       30       30        5
21845       22       16       22       15       29        6       29        6

[21846 rows x 8 columns]
- 特徴選択無し [8個の特徴量] -
Accuracy = 0.695
 